In [1]:
import numpy as np
from families import Gaussian, Bernoulli, Poisson
from glm import GLM

import statsmodels.api as sm
import statsmodels

/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
N = 10000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = np.random.uniform(size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

## Linear Model

In [3]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [4]:
model.coef_

array([ 1.04639307, -2.06157582,  0.97999773])

In [5]:
model.parameter_covariance_

array([[  7.10679492e-04,  -6.01005251e-04,  -6.09652804e-04],
       [ -6.01005251e-04,   1.20289039e-03,  -6.15741739e-06],
       [ -6.09652804e-04,  -6.15741739e-06,   1.21617010e-03]])

In [6]:
np.sqrt(np.diag(model.parameter_covariance_))

array([ 0.02665857,  0.03468271,  0.03487363])

In [7]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.301
Model:                            OLS   Adj. R-squared:                  0.301
Method:                 Least Squares   F-statistic:                     2153.
Date:                Sun, 27 Aug 2017   Prob (F-statistic):               0.00
Time:                        09:54:28   Log-Likelihood:                -14271.
No. Observations:               10000   AIC:                         2.855e+04
Df Residuals:                    9997   BIC:                         2.857e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0464      0.027     39.252      0.0

## Linear Model with Sample Weights

In [8]:
sample_weights = np.random.uniform(0, 2, size=N)

In [9]:
model = GLM(family=Gaussian())
model = model.fit(X, y, sample_weights=sample_weights)

In [10]:
model.coef_

array([ 1.04981231, -2.03580717,  0.95051409])

## Logistic Model

In [11]:
p = 1 / (1 + np.exp(-nu))
y_logistic = np.random.binomial(1, p=p, size=N)

In [12]:
model = GLM(family=Bernoulli())
model.fit(X, y_logistic)

In [13]:
model.coef_

array([ 0.99592445, -1.95687944,  0.90681896])

In [14]:
model.dispersion_

array(1.0)

In [15]:
model.parameter_covariance_

array([[ 0.00324264, -0.00291278, -0.00250159],
       [-0.00291278,  0.00580432, -0.00039687],
       [-0.00250159, -0.00039687,  0.00553404]])

In [16]:
np.sqrt(np.diag(model.parameter_covariance_) / model.dispersion_)

array([ 0.05694421,  0.07618606,  0.0743911 ])

In [17]:
mod = sm.Logit(y_logistic, X)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.628242
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Sun, 27 Aug 2017   Pseudo R-squ.:                 0.06309
Time:                        09:54:28   Log-Likelihood:                -6282.4
converged:                       True   LL-Null:                       -6705.5
                                        LLR p-value:                1.873e-184
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9959      0.057     17.489      0.000       0.884       1.108
x1            -1.9569      0.

In [18]:
coefs = np.empty((1000, 3))
for i in range(1000):
    y_logistic = np.random.binomial(1, p=p, size=N)
    model = GLM(family=Bernoulli())
    model.fit(X, y_logistic)
    coefs[i, :] = model.coef_
    
print(coefs.std(axis=0))

[ 0.05619233  0.07619736  0.07441136]


## Poission Model

In [19]:
mu = np.exp(nu)
y_poisson = np.random.poisson(lam=mu, size=N)

In [20]:
model = GLM(family=Poisson())
model.fit(X, y_poisson)

In [21]:
model.coef_

array([ 1.00067027, -1.98164552,  1.00328601])

In [22]:
model.parameter_covariance_

array([[  3.43663309e-04,  -2.37206319e-04,  -3.63957721e-04],
       [ -2.37206319e-04,   6.92723949e-04,  -2.42282082e-06],
       [ -3.63957721e-04,  -2.42282082e-06,   6.23517757e-04]])

In [23]:
np.sqrt(np.diag(model.parameter_covariance_))

array([ 0.01853816,  0.02631965,  0.02497034])

In [24]:
mod = statsmodels.discrete.discrete_model.Poisson(y_poisson, X)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 1.597049
         Iterations 7
                          Poisson Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        Poisson   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Sun, 27 Aug 2017   Pseudo R-squ.:                  0.1970
Time:                        09:54:33   Log-Likelihood:                -15970.
converged:                       True   LL-Null:                       -19888.
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0007      0.019     53.979      0.000       0.964       1.037
x1            -1.9816      0.

## Poisson with Exposures

In [25]:
mu = np.exp(nu)
expos = np.random.uniform(0, 10, size=N)
y_poisson = np.random.poisson(lam=(mu*expos), size=N)

In [26]:
model = GLM(family=Poisson())
model.fit(X, y_poisson, offset=np.log(expos))

In [27]:
model.coef_

array([ 0.99069216, -1.98004921,  1.00209306])

## Linear Model with Correlated Predictors

In [28]:
N = 1000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = 0.9*X[:, 1] + np.random.uniform(-0.1, 0.1, size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

In [29]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [30]:
model.coef_

array([ 1.06193498, -1.07423537, -0.04581364])

In [31]:
model.parameter_covariance_

array([[ 0.00388138, -0.00351601, -0.0024858 ],
       [-0.00351601,  0.24019843, -0.25807862],
       [-0.0024858 , -0.25807862,  0.29094417]])